# Implementing MiniGoogleNet
- We use tensorflow.keras.layers.Model instead of Sequential because Model will allow us to create a network graph with splits and forks like in Inception module.
- We supply the input layer in parenthesis at the end of the function call, which is called a Functional API,like
$$output = Layer(parameters)(input)$$

In [2]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

In [ ]:
class MiniGoogleNet:
  
  # x: input layer to function
  # K: number of filters our CONV layer is going to learn
  # (kX, kY): size of each K filters that will be learned
  # stride: stride of CONV layer
  # chanDim: The channel dimension, either "channels last" or "channels first"
  # padding: type of padding to CONV layer
  @staticmethod
  def conv_module(x, K, kX, kY, stride, chanDim, padding="same"):
    # define a CONV->BN->RELU pattern
    x = layers.Conv2D(K, (kX, kY), strides=stride, padding=padding)(x)
    x = layers.BatchNormalization(axis=chanDim)(x)
    x = layers.Activation("relu")(x)
    
    return x
  
  # x: input layer to the function
  # numK1x1: number of 1x1 CONV filters
  # numK3x3: number of 3x3 CONV filters
  # chanDim: channel along which to concatenate
  @staticmethod
  def inception_module(x, numK1x1, numK3x3, chanDim):
    # define two CONV modules, then concatenate across the channel dimension
    conv_1x1 = MiniGoogleNet.conv_module(x, numK1x1, 1, 1, (1,1), chanDim)
    conv_3x3 = MiniGoogleNet.conv_module(x, numK3x3, 3, 3, (1,1), chanDim)
    x = layers.concatenate([conv_1x1, conv_3x3], axis=chanDim)
    
    return x
  
  
  @staticmethod
  def downsample_module(x, K, chanDim):
    # define the CONV module and POOl then concatenate across
    # channel dimension
    conv_3x3 = MiniGoogleNet.conv_module(x, K, 3, 3, (2,2), chanDim, padding="valid")
    pool = layers.MaxPooling2D((3,3), strides=(2,2))(x)
    x = layers.concatenate([conv_3x3, pool], axis=chanDim)
    return x
  
  @staticmethod
  def build(width, height, depth, classes):
    # init the input shape to be channels first and channels dimensions itelse
    inputShape = (height, width, depth)
    chanDim = -1
    
    if K.image_data_format() == "channels_first":
      inputShape = (depth, height, width)
      chanDim = 1
      
    # define model input and first CONV module
    inputs = layers.Input(shape=inputShape)
    x = MiniGoogleNet.conv_module(inputs, 96, 3, 3, (1,1), chanDim)
    
    # two Inception modules followed by a downsample
    x = MiniGoogleNet.inception_module(x, 32, 32, chanDim)
    x = MiniGoogleNet.inception_module(x, 32, 48, chanDim)
    x = MiniGoogleNet.downsample_module(x, 80, chanDim)
    
    # stack four inception modules followed by a downsample
    x = MiniGoogleNet.inception_module(x, 112, 48, chanDim)
    x = MiniGoogleNet.inception_module(x, 96, 64, chanDim)
    x = MiniGoogleNet.inception_module(x, 80, 80, chanDim)
    x = MiniGoogleNet.inception_module(x, 48, 96, chanDim)
    x = MiniGoogleNet.downsample_module(x, 96, chanDim)
    
    # Two Inception modules followed by global POOL and dropout
    # Applying an average pooling of 7×7 reduces the volume size
    #     to 1 × 1 × 336 and thereby alleviates the need to apply many
    #     dense fully-connected layers – instead, we simply average over
    #     the spatial outputs of the convolution
    x = MiniGoogleNet.inception_module(x, 176, 160, chanDim)
    x = MiniGoogleNet.inception_module(x, 176, 160, chanDim)
    x = layers.AveragePooling2D((7,7))(x)
    x = layers.Dropout(0.5)(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(classes)(x)
    x = layers.Activation("relu")(x)
    
    # create model
    model = layers.Model(inputs, x, name="googlenet")
    return model